In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
player_df = pd.read_csv('../data/new/players_all.csv')
player_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4742 entries, 0 to 4741
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Id        4742 non-null   int64 
 1   No.       4742 non-null   int64 
 2   Name      4742 non-null   object
 3   Age       4742 non-null   int64 
 4   Team      4742 non-null   object
 5   Position  4742 non-null   object
 6   year      4742 non-null   int64 
dtypes: int64(4), object(3)
memory usage: 259.5+ KB


In [3]:
stats1_df = pd.read_csv('../data/new/1xbet_all.csv')
stats1_df = stats1_df.drop(['Team','Age','Position'], axis=1)
stats1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4756 entries, 0 to 4755
Data columns (total 21 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Name      4756 non-null   object 
 1   Apps      4756 non-null   int64  
 2   Mins      4756 non-null   int64  
 3   Goals     4756 non-null   int64  
 4   Assists   4756 non-null   int64  
 5   SpG       4756 non-null   float64
 6   KeyP      4756 non-null   float64
 7   Drb_x     4756 non-null   float64
 8   Fouled    4756 non-null   float64
 9   Off       4756 non-null   float64
 10  Disp      4756 non-null   float64
 11  UnsTch    4756 non-null   float64
 12  Rating    4756 non-null   float64
 13  Tackles   4756 non-null   float64
 14  Inter     4756 non-null   float64
 15  Fouls     4756 non-null   float64
 16  Offsides  4756 non-null   float64
 17  Clear     4756 non-null   float64
 18  Drb_y     4756 non-null   float64
 19  Blocks    4756 non-null   float64
 20  year      4756 non-null   int6

In [4]:
stats2_df = pd.read_csv('../data/new/understat_all.csv')
stats2_df = stats2_df.drop(['No','Team'], axis=1)
stats2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4755 entries, 0 to 4754
Data columns (total 19 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Player       4755 non-null   object 
 1   Apps         4755 non-null   int64  
 2   Min          4755 non-null   int64  
 3   G            4755 non-null   int64  
 4   NPG          4755 non-null   int64  
 5   A            4755 non-null   int64  
 6   xG           4755 non-null   float64
 7   NPxG         4755 non-null   float64
 8   xA           4755 non-null   float64
 9   xGChain      4755 non-null   float64
 10  xGBuildup    4755 non-null   float64
 11  xG90         4755 non-null   float64
 12  NPxG90       4755 non-null   float64
 13  xA90         4755 non-null   float64
 14  xG90+xA90    4755 non-null   float64
 15  NPxG90+xA90  4755 non-null   float64
 16  xGChain90    4755 non-null   float64
 17  xGBuildup90  4755 non-null   float64
 18  year         4755 non-null   int64  
dtypes: flo

In [5]:
salary_df = pd.read_csv('../data/new/salary_all0.csv')
salary_df = salary_df.drop(['Position', 'Team'], axis=1)
salary_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Rank           4803 non-null   float64
 1   Name           4803 non-null   object 
 2   Weekly Salary  4803 non-null   int64  
 3   year           4803 non-null   int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 150.2+ KB


In [6]:
salary2_df = pd.read_csv('../data/new/capology_all.csv')
salary2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6140 entries, 0 to 6139
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   name     6140 non-null   object
 1   money-w  6140 non-null   int64 
 2   money-y  6140 non-null   int64 
 3   money-a  6140 non-null   int64 
 4   team     6140 non-null   object
 5   year     6140 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 287.9+ KB


In [7]:
# player_df, salary_df, stats1_df, stats2_df
eda_df = player_df.merge(salary2_df[['money-w','money-y','money-a','name', 'year']], how='left', left_on=['Name', 'year'], right_on=['name', 'year'])
eda_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4938 entries, 0 to 4937
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Id        4938 non-null   int64  
 1   No.       4938 non-null   int64  
 2   Name      4938 non-null   object 
 3   Age       4938 non-null   int64  
 4   Team      4938 non-null   object 
 5   Position  4938 non-null   object 
 6   year      4938 non-null   int64  
 7   money-w   4076 non-null   float64
 8   money-y   4076 non-null   float64
 9   money-a   4076 non-null   float64
 10  name      4076 non-null   object 
dtypes: float64(3), int64(4), object(4)
memory usage: 424.5+ KB


In [8]:
eda_df.isnull().sum(axis=0)

Id            0
No.           0
Name          0
Age           0
Team          0
Position      0
year          0
money-w     862
money-y     862
money-a     862
name        862
dtype: int64

In [9]:
eda_df.isnull().mean()

Id          0.000000
No.         0.000000
Name        0.000000
Age         0.000000
Team        0.000000
Position    0.000000
year        0.000000
money-w     0.174565
money-y     0.174565
money-a     0.174565
name        0.174565
dtype: float64

In [10]:
na_list = eda_df.loc[eda_df.isnull().any(axis=1)].loc[:, ['Name','year']]
na_list.groupby('year')['Name'].count()

year
2014     86
2015     81
2016     71
2017     79
2018     80
2019    113
2020    105
2021    121
2022    126
Name: Name, dtype: int64

In [11]:
player_df.groupby('year')['Name'].count()

year
2014    535
2015    547
2016    529
2017    516
2018    497
2019    511
2020    519
2021    533
2022    555
Name: Name, dtype: int64

In [12]:
eda_df.groupby('year')['Name'].count()

year
2014    572
2015    573
2016    568
2017    543
2018    511
2019    520
2020    531
2021    546
2022    574
Name: Name, dtype: int64

In [13]:
eda_df[eda_df['Weekly Salary'].isnull()].head()

KeyError: 'Weekly Salary'

In [ ]:
salary_df.groupby('year')['Name'].count()

In [ ]:
# player_df, salary_df, stats1_df, stats2_df
eda_df = player_df.merge(stats2_df, how='outer', left_on=['Name', 'year'], right_on=['Player', 'year'])
eda_df.info()

In [ ]:
na_df = eda_df[eda_df.isnull().any(axis=1)]
na1 = na_df[na_df['Apps'].isnull()]
na1[['Name', 'year']].to_excel('../data/dummy/na1.xlsx')

In [ ]:
na2 = na_df[~na_df['Apps'].isnull()]
na2[['Player','year']].to_excel('../data/dummy/na2.xlsx')